In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

In [3]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.173 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.2/78.2 GB disk)


In [4]:
!cp -r /content/drive/MyDrive/v8_format/* .

In [ ]:
!rsync -a /content/drive/MyDrive/v8_format .

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=data.yaml epochs=50 imgsz=640 batch=8 project=/content/drive/MyDrive/ball_output name=Ball_Detection

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=data.yaml epochs=50 imgsz=640 batch=8 project=/content/drive/MyDrive/ball_output name=Ball_Detection_1 optimizer='AdamW' close_mosaic=10 cos_lr=True degrees: 0.2 shear: 0.2 perspective: 0.01 mixup: 0.3 warmup_epochs=1.0

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/output/ball_output/weights/best.pt conf=0.55 source=/content/drive/MyDrive/v8_format/images/val/002af65597d93000.jpg save=true